## GPT

### gpt아키텍처  
GPT:트랜스포머의 디코더만 사용  
특징:  
- bert와 달리 이전 단어들로부터 다음 단어를 예측하는 모델이므로 다음 단어를 지속적으로 생성할 수 있어 기본적으로 글쓰기가 가능한 생성 모델  

GPT-1,GPT-2,GPT-3는 아키텍처상 큰 차이가 없음.

## gpt2를 이용한 문장생성  

In [1]:
!pip install transformers

### KoGPT-2로 문장 생성하기  

In [2]:
import numpy as np
import torch
import random
import time
from transformers import AutoTokenizer
from transformers import GPT2LMHeadModel

In [3]:
model=GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")
tokenizer=AutoTokenizer.from_pretrained("skt/kogpt2-base-v2")

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

In [4]:
sent="아침에 눈이 와서"
input_ids=tokenizer.encode(sent,return_tensors="pt")
print(input_ids)


tensor([[42129, 16640, 13300]])


In [5]:
output=model.generate(input_ids,max_length=128,repetition_penalty=2.0,use_cache=True)
output_ids=output.numpy().tolist()[0]
print(output_ids)

[42129, 16640, 13300, 9847, 19909, 9673, 9409, 7220, 10078, 8146, 7623, 9052, 9563, 9325, 9366, 15466, 14705, 24703, 9243, 29213, 7071, 8149, 9168, 10281, 12174, 10473, 6824, 14325, 9829, 15576, 19704, 8263, 6872, 6889, 26764, 15118, 9030, 9022, 15712, 9279, 7235, 10005, 9023, 11763, 35902, 16307, 10091, 15679, 13814, 13099, 26421, 387, 9094, 16373, 14769, 24475, 9916, 9315, 31251, 10675, 13459, 16249, 8704, 10135, 7194, 9326, 7588, 7478, 14897, 10576, 11759, 50451, 9304, 49436, 473, 9515, 12419, 20308, 14585, 17955, 10010, 7335, 10070, 10939, 9148, 9079, 7407, 23706, 9435, 22897, 6903, 25893, 8022, 9183, 24655, 17152, 41776, 10982, 7162, 33114, 9846, 14694, 10542, 37770, 10141, 9517, 15245, 25521, 9564, 32431, 10590, 30899, 9960, 33228, 29554, 13899, 10656, 10897, 12477, 8066, 12094, 10373, 9758, 48101, 9518, 11968, 9312, 33770]


In [6]:
tokenizer.decode(output_ids)

'아침에 눈이 와서 눈발이 날리는데요.\n이번에는 서울에도 많은 양의 비가 내릴 것으로 보입니다.\n내일까지 최고 80mm가 넘는 폭우가 쏟아지겠고요,\n서울 등 그 밖의 지역도 30에서 60mm의 적지 않은 양이 예상됩니다.\n특히, 내일은 중부지방을 중심으로 돌풍과 벼락을 동반한 시간당 20밀리미터 이상의 강한 비와 함께 천둥· 번개가 치는 곳이 있겠습니다.\n또 오늘 밤부터 모레 새벽 사이 서해안과 남해안에 안개와 짙은 안개 끼는 곳도 있을 전망입니다.\n지금 전국 대부분 지방에 구름이 많이 끼어 있습니다.\n오늘 낮 기온은 서울과 대전 24도, 광주와 대구 25도로 어제보다 조금 높겠지만,'

### Numpy로 top5뽑기  

In [8]:
output=model(input_ids)
logits=output.logits[0,-1]
top5=torch.topk(logits,k=5)
tokens=[tokenizer.decode(token_id) for token_id in top5.indices.tolist()]
print(tokens)

['눈이', '눈', '눈을', '그런', '']


### NUMPY로 top5문장 생성하기  

In [9]:
sent="눈이 오는 날에는 따뜻한 코코아를 "
input_ids=tokenizer.encode(sent,return_tensors="pt")
while len(input_ids[0])<50:
  with torch.no_grad():
    output=model(input_ids)
  logits=output.logits[0,-1]
  top5=torch.topk(logits,k=30)
  token_id=random.choice(top5.indices.tolist())
  input_ids=torch.cat([input_ids,torch.tensor([[token_id]])],dim=1)
tokenizer.decode(input_ids[0])

'눈이 오는 날에는 따뜻한 코코아를 쓱 먹어요. 그립도 너무 부드러운 거 맞아 어요!</d> 내 피부 상태에 잘 맞지 않다고 걱정하거나 관리를 게을리 한다면 이미 주름으로 발전되었던 것을 말해도 과언이 되지 못 하건가?\n실내는'

## GPT2를 이용한 한국어 뉴스 긍정, 부정 감성분류  

In [1]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_syst

### 데이터로드

In [3]:
!pip install tqdm

In [6]:
from datasets import load_dataset
import pandas as pd
import numpy as np
import datetime
import tqdm
import torch
from transformers import BertTokenizer
from transformers import AutoTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,roc_auc_score,accuracy_score

In [7]:
!wget https://raw.githubusercontent.com/ukairia777/finance_sentiment_corpus/main/finance_data.csv

--2025-01-30 12:53:48--  https://raw.githubusercontent.com/ukairia777/finance_sentiment_corpus/main/finance_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1319001 (1.3M) [text/plain]
Saving to: ‘finance_data.csv’

finance_data.csv    100%[===================>]   1.26M  --.-KB/s    in 0.01s   

2025-01-30 12:53:49 (116 MB/s) - ‘finance_data.csv’ saved [1319001/1319001]



In [8]:
df=pd.read_csv("finance_data.csv")
df.head()

,labels,sentence,kor_sentence
0,neutral,"According to Gran, the company has no plans to...","Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,neutral,Technopolis plans to develop in stages an area...,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,negative,The international electronic industry company ...,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,positive,With the new production plant the company woul...,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,positive,According to the company's updated strategy fo...,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."


In [9]:
df['labels'].value_counts()

,count
labels,
neutral,2879
positive,1363
negative,604


In [10]:
df['labels']=df['labels'].replace(['neutral','positive','negative'],[0,1,2])
df.head()

<ipython-input-10-2af4c1ea7305>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['labels']=df['labels'].replace(['neutral','positive','negative'],[0,1,2])


,labels,sentence,kor_sentence
0,0,"According to Gran, the company has no plans to...","Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,0,Technopolis plans to develop in stages an area...,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,2,The international electronic industry company ...,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,1,With the new production plant the company woul...,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,1,According to the company's updated strategy fo...,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."


In [11]:
df.to_csv("finance_data.csv",index=False,encoding='utf-8-sig')

In [12]:
all_data=load_dataset(#dataset패키지의 load_dataset을 이용해서 ㅇatasetdic객체를 로드가능
    "csv",
    data_files={
        "train":'finance_data.csv',
    }
)

Generating train split: 0 examples [00:00, ? examples/s]

In [19]:
cs=all_data['train'].train_test_split(0.2,seed=777)
train_cs=cs['train']
test_cs=cs['test']

In [20]:
train_cs

Dataset({
    features: ['labels', 'sentence', 'kor_sentence'],
    num_rows: 3876
})

In [21]:
test_cs

Dataset({
    features: ['labels', 'sentence', 'kor_sentence'],
    num_rows: 970
})

In [22]:
cs=train_cs.train_test_split(0.2,seed=777)#훈련데이터를 다시 8:2로 나눠서 훈련데이터, 검증데이터로 설정
train_cs=cs['train']
valid_cs=cs['test']

In [23]:
train_cs

Dataset({
    features: ['labels', 'sentence', 'kor_sentence'],
    num_rows: 3100
})

In [24]:
valid_cs

Dataset({
    features: ['labels', 'sentence', 'kor_sentence'],
    num_rows: 776
})

In [25]:
test_cs

Dataset({
    features: ['labels', 'sentence', 'kor_sentence'],
    num_rows: 970
})

In [26]:
print(train_cs['kor_sentence'][1])
print(train_cs['labels'][1])

알마 미디어 코퍼레이션 사업 ID 1944757-4의 주식 자본금은 44,767,513.80유로이며 74,612,523주로 나뉜다.
0


### 데이터 전처리  

In [27]:
train_sentences=list(train_cs["kor_sentence"])
validation_sentences=list(valid_cs["kor_sentence"])
test_sentences=list(test_cs["kor_sentence"])

In [28]:
train_labels = train_cs['labels']
validation_labels = valid_cs['labels']
test_labels = test_cs['labels']

In [29]:
test_sentences[:5]

['오전 10.58시 아우토쿰푸는 2.74pct 하락한 24.87유로, OMX 헬싱키 25지수는 0.55pct 상승한 2,825.14, OMX 헬싱키는 0.64pct 하락한 9,386.89유로에 거래됐다.',
 '10월부터 12월까지의 판매량은 302 mln 유로로 전년 동기 대비 25.3 pct 증가했다.',
 '매디슨, 위스콘신, 2월 6일 - PRNewswire - - 피스카스는 미국 특허청이 상징적인 가위 손잡이에 오렌지색 상표 등록을 허가했다고 발표한다.',
 "M-real로 평가된 분석가들 중 총 6명은 ''매수' - ''누적''을 주었고, 3명은 ''보유'', 1명만이 ''매도''를 주었다.",
 '주요 양조업체들은 지난해 국내 맥주 판매량을 2004년 2억4592만 리터에서 2억5688만 리터로 4.5% 늘렸다.']

In [31]:
test_labels[:5]

[0, 1, 1, 0, 1]

### GPT토크나이저를 이용한 전처리  

In [35]:
tokenizer=AutoTokenizer.from_pretrained("skt/kogpt2-base-v2")

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

In [36]:
MAX_LEN=128
def data_to_tensor(sentences,labels):
  tokenized_texts=[tokenizer.tokenize(sent) for sent in sentences]
  input_ids=[tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
  pad_token=tokenizer.encode("<pad>")[0]#주어진 최대길이를 위해서 패딩토큰의 번호로 채워준다
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, value=pad_token, dtype="long", truncating="post", padding="post")
  attention_masks=[]
  for seq in input_ids:
    seq_mask=[float(i!=pad_token) for i in seq]
    attention_masks.append(seq_mask)
  tensor_inputs=torch.tensor(input_ids)
  tensor_labels=torch.tensor(labels)
  tensor_masks=torch.tensor(attention_masks)
  return tensor_inputs,tensor_labels,tensor_masks

정수인코딩 과정:각 텍스트트 토큰화한 후, vocabulary에 mapping되는 정수시퀀스로 변환  
pad_sequences는 패딩위한 모듈.주어진 최대 길이를 위해서 뒤에서 패딩 토큰의 번호로 채운다  
즉, 위 함수를 통해 정수인코딩데이터, 레이블데이터, 어텐션 마스크를 return

In [37]:
max_len=128

In [44]:
train_inputs, train_labels, train_masks = data_to_tensor(train_sentences, train_labels)
validation_inputs,validation_labels,validation_masks=data_to_tensor(validation_sentences,validation_labels)
test_inputs, test_labels, test_masks =data_to_tensor(test_sentences, test_labels)

### 데이터의 배치화  

데이터로더를 이용하여 데이터를 배치단위로 학습  

In [45]:
batch_size=32
train_data=TensorDataset(train_inputs,train_masks,train_labels)
train_sampler=RandomSampler(train_data)
train_dataloader=DataLoader(train_data,sampler=train_sampler,batch_size=batch_size)

batch_size=모델 학습시 한번에 처리할 데이터수  
TensorDataset=텐서를 입력받아 데이터셋 만듦  
RandomSampler=데이터셋에서 무작위로 데이터 추출하는샘플러(과적합 방지)  
DataLoader=데이터셋과 샘플러를 입력으로 받아 배치단위로 데이터를 모델에 공급하는 역할  

In [46]:
validation_data=TensorDataset(validation_inputs,validation_masks,validation_labels)
validation_sampler=SequentialSampler(validation_data)
validation_dataloader=DataLoader(validation_data,sampler=validation_sampler,batch_size=batch_size)

In [47]:
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [48]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [50]:
from transformers import AutoModelForSequenceClassification

In [51]:
num_labels=3
model=AutoModelForSequenceClassification.from_pretrained("skt/kogpt2-base-v2",num_labels=num_labels)
model.cuda()

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at skt/kogpt2-base-v2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=3, bias=False)
)

### 모델 학습

In [52]:
epochs = 3
optimizer = AdamW(model.parameters(), lr = 2e-5)

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [53]:
def metrics(predictions,labels):
  y_pred=predictions
  y_true=labels
  accuracy=accuracy_score(y_true,y_pred)
  f1_macro_avg=f1_score(y_true=y_true,y_pred=y_pred,average='macro',zero_division=0)
  f1_micro_avg=f1_score(y_true=y_true,y_pred=y_pred,average='micro',zero_division=0)
  f1_weighted_average = f1_score(y_true=y_true, y_pred=y_pred, average='weighted', zero_division=0)
  #계산된 메트릭 결과를 딕셔너리 형태로 리턴
  metrics = {'accuracy': accuracy,
               'f1_macro': f1_macro_avg,
               'f1_micro': f1_micro_avg,
               'f1_weighted': f1_weighted_average}
  return metrics

In [61]:
from tqdm import tqdm
def train_epoch(model, train_dataloader, optimizer, device):
    """
    하나의 에포크 동안 모델을 학습시키는 함수입니다.

    Parameters:
    model (torch.nn.Module): 학습시킬 모델 객체.
    train_dataloader (torch.utils.data.DataLoader): 학습 데이터셋의 DataLoader.
    optimizer (torch.optim.Optimizer): 최적화 알고리즘을 구현하는 객체.
    device (torch.device): 학습에 사용할 장치(CPU 또는 CUDA).

    Returns:
    float: 평균 학습 손실값.
    """
    total_train_loss=0  #학습 손실을 누적할 변수 초기화
    model.train() #모델을 학습 모드로 설정
    #학습 데이터로더를 순회하며 배치 단위로 학습
    for step, batch in tqdm(enumerate(train_dataloader),desc="Training Batch"):
        batch=tuple(t.to(device) for t in batch)  #DataLoader에서 배치를 받아 각 텐서를 지정된 장치로 이동
        b_input_ids, b_input_mask, b_labels=batch  #배치에서 입력 ID, 마스크, 라벨 추출
        #모델에 배치를 전달하여 손실값 계산
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        #손실값 추출
        loss=outputs.loss
        optimizer.zero_grad()  #기울기(gradient) 초기화
        loss.backward()  #역전파를 통해 기울기(gradient) 계산
        optimizer.step()  #가중치 매개변수 업데이트
        total_train_loss+=loss.item()
    avg_train_loss=total_train_loss/len(train_dataloader)
    return avg_train_loss

train_epoch()함수:함수의 입력으로 학습시킬 모델 객체, 학습데이터셋의 DataLoader,최적화 알고리즘 구현하는 옵티마이저객체  

cf)학습 손실을 누적할 변수 초기화하고 모델을 학습모드로 설정, 학습데이터로더를 순회하면서 배치단위로 학습진행  
cf)각 배치에 대해 모델에 입력하여 손실값 계산  
(이때 입력데이터->입력id,마스크,라벨로 구성됨)  
역전차로 기울기 계산 및 모델 매개변수 업데이트  
cf)한 epoch학습이 끝나면 총 손실을 배치개수로 나눠서 평균 손실 계산  

In [64]:
def evaluate(model,validation_dataloader,device):
  model.eval()
  total_eval_loss=0#검증손실을 누적할 변수 초기화
  predictions,true_labels=[],[]
  for batch in validation_dataloader:
    batch=tuple(t.to(device) for t in batch)#배치데이터를 디바이스로 이동
    b_input_ids,b_input_mask,b_labels=batch#배치에서 입력id,마스크,라벨 추출
    with torch.no_grad():
      outputs=model(b_input_ids,token_type_ids=None,attention_mask=b_input_mask,labels=b_labels)
    if outputs.loss is not None:
      loss=outputs.loss
      total_eval_loss+=loss.item()
      logits = outputs.logits.detach().cpu().numpy()
      label_ids = b_labels.to('cpu').numpy()
      #3개의 값 중 가장 큰 값을 예측한 인덱스로 결정
      predictions.extend(np.argmax(logits, axis=1).flatten()) #예측된 클래스를 리스트에 추가
      true_labels.extend(label_ids.flatten()) # 실제 레이블 값을 리스트에 추가
  eval_metrics = metrics(predictions, true_labels)
  return total_eval_loss / len(validation_dataloader), eval_metrics


In [65]:
min_val_loss=float('inf')
for epoch_i in range(0,epochs):
  print("===========Epoch{:}/{:}===========".format(epoch_i+1,epochs))
  train_epoch(model,train_dataloader,optimizer,device)
  print("검증 셋 하느 중")
  avg_val_loss,eval_metrics=evaluate(model,validation_dataloader,device)
  print("  Validation Loss: {0:.2f}".format(avg_val_loss))
  print("  Accuracy: {0:.2f}".format(eval_metrics['accuracy']))
  print("  F1 Macro: {0:.2f}".format(eval_metrics['f1_macro']))
  print("  F1 Micro: {0:.2f}".format(eval_metrics['f1_micro']))
  print("  F1 Weighted: {0:.2f}".format(eval_metrics['f1_weighted']))
  if avg_val_loss<min_val_loss:
    min_val_loss=avg_val_loss
    print(f"Validation loss decreased ({min_val_loss:.2f} --> {avg_val_loss:.2f}).  Saving model ...")
    #베스트 모델 저장
    torch.save(model.state_dict(), 'model_checkpoint.pt')

===========Epoch1/3===========


Training Batch: 97it [01:08,  1.42it/s]


검증 셋 하느 중
  Validation Loss: 0.43
  Accuracy: 0.84
  F1 Macro: 0.79
  F1 Micro: 0.84
  F1 Weighted: 0.83
Validation loss decreased (0.43 --> 0.43).  Saving model ...
===========Epoch2/3===========


Training Batch: 97it [01:07,  1.43it/s]


검증 셋 하느 중
  Validation Loss: 0.52
  Accuracy: 0.81
  F1 Macro: 0.78
  F1 Micro: 0.81
  F1 Weighted: 0.81
===========Epoch3/3===========


Training Batch: 97it [01:07,  1.43it/s]


검증 셋 하느 중
  Validation Loss: 0.58
  Accuracy: 0.83
  F1 Macro: 0.80
  F1 Micro: 0.83
  F1 Weighted: 0.83


### 모델 로드 및 평가  

In [66]:
# 베스트 모델 로드
model.load_state_dict(torch.load("model_checkpoint.pt"))

avg_val_loss, eval_metrics = evaluate(model, test_dataloader, device)
print("  Test Loss: {0:.2f}".format(avg_val_loss))
print("  Accuracy: {0:.2f}".format(eval_metrics['accuracy']))
print("  F1 Macro: {0:.2f}".format(eval_metrics['f1_macro']))
print("  F1 Micro: {0:.2f}".format(eval_metrics['f1_micro']))
print("  F1 Weighted: {0:.2f}".format(eval_metrics['f1_weighted']))

<ipython-input-66-c4493010d792>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model_checkpoint.pt"))


  Test Loss: 0.44
  Accuracy: 0.83
  F1 Macro: 0.79
  F1 Micro: 0.83
  F1 Weighted: 0.82


### 추론하기

Pipeline:예측함수 자동으로 만들어줌

In [67]:
from transformers import pipeline
pipe = pipeline("text-classification", model=model.cuda(), tokenizer=tokenizer, device=0, max_length=512, return_all_scores=True, function_to_apply='softmax')

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [68]:
result = pipe('SK하이닉스가 매출이 급성장하였다')
print(result)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[[{'label': 'LABEL_0', 'score': 0.03629642724990845}, {'label': 'LABEL_1', 'score': 0.963074803352356}, {'label': 'LABEL_2', 'score': 0.0006288372678682208}]]


In [69]:
#return_all_scores 제거-->정답인 카테고리만 출력
pipe = pipeline("text-classification", model=model.cuda(), tokenizer=tokenizer, device=0, max_length=512, function_to_apply='softmax')
result = pipe('SK하이닉스가 매출이 급성장하였다')
print(result)

Device set to use cuda:0


[{'label': 'LABEL_1', 'score': 0.963074803352356}]


In [72]:
label_dict = {'LABEL_0' : '중립', 'LABEL_1' : '긍정', 'LABEL_2' : '부정'}
def prediction(text):
  result = pipe(text)
  return [label_dict[result[0]['label']]]

In [73]:
prediction('인공지능 기술의 발전으로 누군가는 기회를 얻을 것이고, 누군가는 얻지 못할 것이다')

['중립']